# RSA

Este Notebook sobre encripción RSA es una combinación del código habitual para realizar este tipo de encriptación acompañado de la descripción correspondiente de cada bloque. Puedes ejecutar cada sección seleccionandola y presionando ```RUN``` en la parte superior de esta pantalla o simplemente selecionando ```RUN ALL```desde el menún ```Cell```.

## Prerrequisitos

In [24]:
import random  # Vamos a necesitar números aleatorios
import math    # También vamos a necesitar algunas funciones matemáticas

# Inverso multiplicativo de k en los enteros módulo p, con p preferentemente primo para garantizar unicidad.
def modInverse(k, p):
    k = k % p
    for x in range(1, p):
        if ((k * x) % p == 1):
            return x
    return 1

Si estás familiarizado con la función módulo para formar los campos $Z_p$ (los enteros módulo $p$ con $p$ primo) entonces solo necesitas saber que ```modInverse(k, p)``` calcula el inverso multiplicativo de $k$ dentro del campo $Z_p$. Si lo que te acabo de decir no tiene sentido, entonces lee los siguientes párrafos:

### Módulos
> Pensemos brevemente en las horas del reloj, estas van de la hora 1 a la hora 12 para luego repetirse. Después del 12, el siguiente número que debería ser el 13 se vuelve a representar como el 1, el 14 como el 2 y así sucesivamente. Formalmente decimos que el número 13 es congruente con 1 módulo 12 y que 14 es congruente con 2 módulo 12. El 15 sería entonces congruente con 3 (módulo 12) lo cual se denota de la siguiente manera: $$15 \equiv 3 (mod 12)$$ $$ó$$ $$15\%12 = 3$$Lo cual se lee **15 es equivalente con 3 modulo 12**, ó **15 módulo 12 es igual a 3**.
> El ciclo se repite haciendo que en algún momento 27 también sea congruente con 3 (módulo 12), haciendo un poco las cuentas puedes ver que los números 3, 15, 27 y 39 son todos 3 módulo 12.

### Inversos en $Z_p$
> Ahora vamos con los inversos. El inverso multiplicativo de $A$ es aquel número $B$ tal que $A \times B = 1$. En el campo de los números reales el inverso de $A$ es $\frac{1}{A}$ ya que $A \times \frac{1}{A} = \frac{A}{A} = 1$, pero en los enteros módulo p es diferente. Por ejemplo en los enteros módulo $3$, $2 \times 2 = 4$ pero $4$ es equivalente a $1$ módulo $3$, por lo tanto $2$ es su propio inverso. En $Z_5$ el $2$ tiene como inverso al $3$ y visceversa, y el $4$ es su propio inverso al igual que el $1$.

### Función $\phi$ de Euler
> La funcion $\phi$ de Euler es una función importante en teoría de números que va de los naturales en los naturales. $\phi(n)$ se define como la cantidad de naturales menores que $n$ que son primos relativos con él. Euler probó un teorema que lleva su nombre que dice que si $a$ y $n$ son primos relativos, entonces $a^{\phi (n)} \equiv 1$ $(mod$ $n)$.

In [18]:
[
modInverse(2,3),  # Obtenemos el inverso del 2 módulo 3
modInverse(2,5),  # Inverso de 2 módulo 5
modInverse(4,5),  # Inverso de 4 módulo 5
modInverse(modInverse(2,5),5),  # Inverso del inverso de 2 módulo 5
modInverse(7,40)
]

[2, 3, 4, 2, 23]

## Preparando al algoritmo RSA
Ahora sí, para comenzar el algoritmo solo debemos escoger 2 números primos $p$ y $q$ y multiplicarlos para obtener lo que llamarémos **módulo público**: 

In [20]:
p = 3
q = 5
moduloPublico = p * q

Ahora calcularemos la cantidad de primos relativos entre 1 y el **módulo público** que es el producto de $(p-1)(q-1)$. A este número lo llamarémos $phi$ por la función de Euler mencionada más arriba.

In [21]:
phi = (p-1)*(q-1)
print("Módulo público: {}    phi: {}".format(moduloPublico, phi))

Módulo público: 15    phi: 8


Ahora tomaremos un número $expPub$ que será parte de nuestra llave pública. $expPub$ debe estar entre 1 y $phi$ y debe ser primo relativo con $phi$: 

In [22]:
expPub = random.randrange(1, phi)
while (math.gcd(expPub, phi) != 1):    # Si el gcd (maximo común divisor) es 1, entonces son primos relativos.
    expPub = random.randrange(1, phi)
print("expPub: ", expPub)

expPub:  7


In [ ]:
expPub = expPub   # Esta línea es por si quieres elegir tu propio exp.

Por último necesitamos calcular el inverso modulo $phi$ de $expPub$, para obtener nuestro exponente privado al que llamarémos 
$expPriv$:

In [27]:
expPriv = modInverse(expPub, phi)
print("expPriv: ", expPriv)

expPriv:  7


### Obteniendo la llave pública y la llave privada
Una vez que tenemos nuestros primos $p$ y $q$, nuestros **módulo público** y nuestros **exponentes público** y **privado** podemos crear nuestras llave pública y nuestra llave privada. 

La llave pública se conforma por el **módulo público** y por el **exponente público**.

La llave privada se conforma por $p$, $q$ y el **exponente privado**.

In [28]:
print("Llave pública: ({}, {})".format(moduloPublico, expPub))

Llave pública: (15, 7)


In [29]:
print("Llave privada: ({}, {}, {})".format(expPriv, p, q))

Llave privada: (7, 3, 5)


### ¡A encriptar!
Una vez que tenemos la llave pública, podemos difundirla para que cualquier persona pueda encriptar mensajes que solo nosotros podamos desencriptar.

Supongamos que alguien quiere encriptar un número, su PIN bancario por ejemplo, que en este caso va a ser la clave siempre infalible $123$.

In [33]:
clave = 12
print ("La clave es: ", clave)
claveEncriptada = (clave ** expPub) % moduloPublico
print ("La clave encriptada es: ", claveEncriptada)

La clave es:  12
La clave encriptada es:  3


### ¡A desencriptar!
Si nosotros fueramos el banco, podemos hacer que cada cajero tenga la llave pública, así puede encriptar el PIN de cada usuario y hacernoslo llegar. En este caso nos lo hace llegar con la clave encriptada. 

Desencriptemos el mensaje:

In [36]:
claveEncriptadaRecibida = 3
claveDesencriptada = (claveEncriptadaRecibida ** expPriv) % (moduloPublico)   # moduloPublico = pq
print ("La clave desencriptada es: ", claveDesencriptada)

La clave desencriptada es:  12
15
15


### ¿Porqué funciona?
Observa que todo el truco está en que exista la operación inversa, lo cual está garantizado por el Teorema de Euler que dice que si $a$ y $n$ son primos relativos, entonces $a^{\phi (n)} \equiv 1$ $(mod$ $n)$.